In [1]:
import pandas as pd
import numpy as np
import urllib2
from bs4 import BeautifulSoup
from unidecode import unidecode
from tqdm import tnrange

# 1.Fetch data and create csv 

In [16]:
site_billboard = 'https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_'

def create_billboard_csv(start_year,end_year):
    billboard_data = []
    for year in xrange(start_year, end_year+1):
        response = urllib2.urlopen(site_billboard + str(year))
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')
        if soup.table:
            table = soup.find('table', attrs={'class':'wikitable sortable'})
            rows = table.findChildren(['tr'])
            for row in rows:
                curr_row = []
                #Add position and year
                song_pos = ((len(billboard_data))%100)+1
                curr_row.extend([song_pos,year])
                cells = row.findChildren('td')
                for cell in cells:
                    #Clean up text
                    text = unidecode(cell.text).replace("\n","").replace("\"","")
                    curr_row.append(text)
                    
                    #Hack to make wiki tables <1982 work correctly
                    if len(curr_row) == 5:
                        del curr_row[2]
                        
                if len(curr_row)==4:
                    #Create Main_Artist feature
                    curr_row.append(text.split("featuring", 1)[0].rstrip())
                    #Add new row to data
                    billboard_data.append(curr_row)

    #Create pandas file from data
    return pd.DataFrame(billboard_data,
                        columns=['Position','Year','Title','Artist','Main Artist'])

create_billboard_csv(1960,1960)

,Position,Year,Title,Artist,Main Artist
0,1,1960,Theme from A Summer Place,Percy Faith,Percy Faith
1,2,1960,He'll Have to Go,Jim Reeves,Jim Reeves
2,3,1960,Cathy's Clown,The Everly Brothers,The Everly Brothers
3,4,1960,Running Bear,Johnny Preston,Johnny Preston
4,5,1960,Teen Angel,Mark Dinning,Mark Dinning
5,6,1960,I'm Sorry,Brenda Lee,Brenda Lee
6,7,1960,It's Now or Never,Elvis Presley,Elvis Presley
7,8,1960,Handy Man,Jimmy Jones,Jimmy Jones
8,9,1960,Stuck on You,Elvis Presley,Elvis Presley
9,10,1960,The Twist,Chubby Checker,Chubby Checker


# 2. Fetch lyrics for saved data

In [43]:
site_lyrics = "http://lyrics.wikia.com/wiki/"

#Using header for requests to avoid 403
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
               'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'
              }

def add_lyrics(dataframe):
    lyric_data=[]
    for i in tnrange(0,dataframe.shape[0],desc='2nd loop',leave=False):
        
        #Forming correct query Arist:Title , 'and' is parsed as '%26' on the site
        query = (site_lyrics+dataframe.loc[i]['Main Artist'].replace(" and "," %26 ")
                 +":"
                 +dataframe.loc[i]['Title']).replace(" ","_")
        try:
            response = urllib2.Request(query,headers=hdr)
            page = urllib2.urlopen(response)
            html = page.read()
            soup = BeautifulSoup(html, 'html.parser')
            lyrics_box= soup.find('div',class_="lyricbox")
            #Replace <br> tags with white space
            for br in lyrics_box.find_all("br"):
                br.replace_with(" ")
            #using lower cases for uniformity
            lyric_data.append(lyrics_box.get_text().lower())
            
        except:
            #if site not found or other error : empty lryics
            lyric_data.append(" ")
            
    dataframe['Lyrics']=lyric_data

# 3. Create output files 

In [48]:
def output_files():
    for i in tnrange(1960,1970,10,desc='1st loop'):
        dataframe = create_billboard_csv(i,i+10)
        add_lyrics(dataframe)
        dataframe.to_csv("data/billboard_top_"+str(i)+"'s.csv", sep='\t', encoding='utf-8')

output_files()